In [1]:
import pandas as pd

In [13]:


# --- Input: your candidates file ---
candidates_file = "output/candidates/new_fxrt_w0.2_candidates.csv"
output_reg = "output/candidates/fxrt_candidates_w0.2_final.reg"

# --- Load candidate table ---
df = pd.read_csv(candidates_file)

# --- Create DS9 region header ---
lines = ["# Region file format: DS9 version 4.1",
         "global color=red font=\"helvetica 10 normal\"",
         "fk5"]

# --- Add one circle per candidate ---
for _, row in df.iterrows():
    ra = float(row["RA"])
    dec = float(row["DEC"])
    obsid = str(row["ObsId"])
    lines.append(f"circle({ra},{dec},2\") # text={{ObsID {obsid}}}")

# --- Write to file ---
with open(output_reg, "w") as f:
    f.write("\n".join(lines))
            
print(f"✅ DS9 region file written to: {output_reg}")
print("💡 Open it in DS9 with: File → Load Regions → fxrt_candidates_w0.2.reg")


KeyError: 'ObsId'

In [20]:
import pandas as pd
import os

# Input file
input_file = 'crossmatches/fxrt_positions_R95.txt'

# Read the table (space/tab delimited)
df = pd.read_csv(input_file, sep=r'\s+')
print("Loaded columns:", df.columns.tolist())

# Define output file path (same folder, same base name but .reg extension)
base, _ = os.path.splitext(input_file)
output_file = base + '_3sigma.reg'

# Write DS9 region file
with open(output_file, 'w') as f:
    f.write('# Region file format: DS9 version 4.1\n')
    f.write('global color=green width=1 font="helvetica 10 bold"\n')
    f.write('fk5\n')
    for _, row in df.iterrows():
        ra = row['RA_CORR']
        dec = row['DEC_CORR']
        r = 1.405*row['R95_arcsec']
        obsid = row['ObsId']
        f.write(f'circle({ra},{dec},{r}") # text={{ObsID {int(obsid)}  R_3sigma={r:.2f}"}}\n')


# Print the absolute path to the region file
abs_path = os.path.abspath(output_file)
print(f"\n✅ Region file saved as:\n{abs_path}")




Loaded columns: ['ObsId', 'RA_CORR', 'DEC_CORR', 'R95_arcsec']

✅ Region file saved as:
/data/tvlieshout/fxrt/search/crossmatches/fxrt_positions_R95_3sigma.reg


In [16]:
with open('crossmatches/fxrt_positions_R95.reg') as f:
    for i, line in enumerate(f):
        print(f"{i+1:3}: {repr(line)}")
        if i > 10:
            break

  1: '# Region file format: DS9 version 4.1\n'
  2: 'global color=green width=1 font="helvetica 10 bold"\n'
  3: 'fk5\n'
  4: 'circle(296.11682,50.582322,0.915611805") # text={ObsID 7439.0  R_3sigma}=0.92"}\n'
  5: 'circle(175.005028,-31.917419,1.24851391") # text={ObsID 9841.0  R_3sigma}=1.25"}\n'
  6: 'circle(193.925584,38.355548,1.161978555") # text={ObsID 9923.0  R_3sigma}=1.16"}\n'
  7: 'circle(296.11682,50.582322,0.915611805") # text={ObsID 7439.0  R_3sigma}=0.92"}\n'
  8: 'circle(175.005028,-31.917419,1.24851391") # text={ObsID 9841.0  R_3sigma}=1.25"}\n'
  9: 'circle(193.925584,38.355548,1.161978555") # text={ObsID 9923.0  R_3sigma}=1.16"}\n'
 10: 'circle(135.38787,32.449891,4.790410725") # text={ObsID 16650.0  R_3sigma}=4.79"}\n'
 11: 'circle(225.774293,-41.685962,1.8546702499999999") # text={ObsID 3838.0  R_3sigma}=1.85"}\n'
 12: 'circle(146.822488,31.855843,0.0") # text={ObsID 6661.0  R_3sigma}=0.00"}\n'


In [6]:
with open('crossmatches/fxrt_positions_R95.reg', 'rb') as f:
    print(f.read(80))

b'# Region file format: DS9 version 4.1\nglobal color=green width=1 font="helvetica'


In [1]:
# create_regions.py

# Define your regions (x, y, radius)
regions = [
    (4104.2082, 2931.9532, 44.703995),
    (4218.213,  2940.6342, 47.143121),
    (4089.7408, 3063.8981, 54.187589)
]

# Create a DS9 region file
with open("bkg.reg", "w") as f:
    f.write("# Region file format: DS9 version 4.1\n")
    f.write("image\n")
    for x, y, r in regions:
        f.write(f"circle({x},{y},{r})\n")

print("Saved bkg.reg")

evt_file = "detections/30892/578_evt2_filtered.fits"     # event file
src_reg_file = "crossmatches/fxrt_source.reg" 

Saved bkg.reg


In [23]:
import re, math
import numpy as np
from astropy.io import fits
from scipy.stats import poisson

# ========== USER INPUTS ==========
evt_file = "detections/30892/578_evt2_filtered.fits"     # event file
src_reg_file = "crossmatches/fxrt_source.reg" 
bkg_reg_file = "bkg.reg"        # background region(s)
exposure_time = 20000.0         # seconds (adjust for your good-time exposure)
psf_fraction = 0.9              # encircled-energy fraction (e.g. 0.9 = 90%)
confidence = 0.9                # confidence level (0.9 = 90%)
pixscale = 0.492                # arcsec/pixel for Chandra ACIS
# =================================

override_source_counts = 23  # <-- put 23 here, or None to auto-measure

# --- 1. Load event file ---
with fits.open(evt_file) as hdul:
    ev = hdul["EVENTS"].data
    cols = [c.lower() for c in ev.columns.names]
    have_xy = all(c in cols for c in ("x", "y"))
    if not have_xy:
        raise RuntimeError("Event file missing X,Y columns (even lowercase).")
    # extract x,y arrays (handles lowercase)
    x = ev[[n for n in ev.columns.names if n.lower()=="x"][0]].astype(float)
    y = ev[[n for n in ev.columns.names if n.lower()=="y"][0]].astype(float)

coords = np.column_stack((x, y))
print(f"Loaded {len(coords)} events.")


# --- 2. Parse DS9 circle() definitions ---
def parse_ds9_circles(filename):
    circles = []
    frame = "image"
    with open(filename) as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#") or line.startswith("global"):
                continue
            if line.lower() in ("image", "fk5"):
                frame = line.lower()
                continue
            if line.startswith("circle("):
                nums = re.findall(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", line)
                if len(nums) >= 3:
                    x0, y0, r = map(float, nums[:3])
                    circles.append((x0, y0, r))
    if len(circles) == 0:
        raise ValueError(f"No circle() definitions found in {filename}")
    if frame != "image":
        print(f"⚠️  Warning: {filename} is in {frame} coordinates, "
              "expected IMAGE. Convert in DS9 if results look wrong.")
    return circles


# --- 3. Count photons in a circular region ---
def count_in_circle(x, y, x0, y0, r_pix):
    dist2 = (x - x0)**2 + (y - y0)**2
    return np.count_nonzero(dist2 <= r_pix**2)


# --- 4. Source counts ---
src = parse_ds9_circles(src_reg_file)[0]  # only one source region
x_src, y_src, r_src_pix = src
measured_N_src = count_in_circle(x, y, x_src, y_src, r_src_pix)
N_src = measured_N_src if override_source_counts is None else int(override_source_counts)
print(f"  photons detected (measured) = {measured_N_src}")
if override_source_counts is not None:
    print(f"  photons used (manual override) = {N_src}")

A_src_arcsec2 = math.pi * (r_src_pix * pixscale) ** 2

print(f"\nSource region:")
print(f"  center = ({x_src:.2f}, {y_src:.2f}) pixels")
print(f"  radius = {r_src_pix:.2f} pixels ({r_src_pix*pixscale:.2f} arcsec)")
print(f"  photons detected = {N_src}")
print(f"  area = {A_src_arcsec2:.2f} arcsec²")


# --- 5. Background counts (can be multiple circles) ---
bkg_regions = parse_ds9_circles(bkg_reg_file)
N_bkg_total = 0
A_bkg_total = 0.0

for (x0, y0, r0) in bkg_regions:
    N = count_in_circle(x, y, x0, y0, r0)
    A = math.pi * (r0 * pixscale) ** 2
    N_bkg_total += N
    A_bkg_total += A

print(f"\nBackground regions:")
print(f"  total photons = {N_bkg_total}")
print(f"  total area    = {A_bkg_total:.1f} arcsec²")


# --- 6. Background surface density and expected counts ---
rho = N_bkg_total / A_bkg_total
b_expected = rho * A_src_arcsec2
print(f"\nBackground surface density ρ = {rho:.6f} counts/arcsec²")
print(f"Expected background in source aperture b = {b_expected:.3f} counts")


# --- 7. Poisson upper limit on net source counts ---
def upper_limit_poisson(N_obs, bkg, conf=0.9):
    # Solve P(N <= N_obs | S+bkg) = 1 - conf for S
    s_vals = np.linspace(0, 50, 50001)
    cdf = poisson.cdf(N_obs, s_vals + bkg)
    idx = np.searchsorted(cdf, 1 - conf)
    return s_vals[idx] if idx < len(s_vals) else np.nan

s_UL = upper_limit_poisson(N_src, b_expected, conf=confidence)
rate_UL = s_UL / (exposure_time * psf_fraction)

print(f"\n{int(confidence*100)}% upper limit on net counts: {s_UL:.3f}")
print(f"PSF-corrected count-rate upper limit: {rate_UL:.3e} counts/s")


Loaded 49120 events.
  photons detected (measured) = 0
  photons used (manual override) = 23

Source region:
  center = (192.74, -6.57) pixels
  radius = 16.36 pixels (8.05 arcsec)
  photons detected = 23
  area = 203.58 arcsec²

Background regions:
  total photons = 116
  total area    = 5442.8 arcsec²

Background surface density ρ = 0.021312 counts/arcsec²
Expected background in source aperture b = 4.339 counts

90% upper limit on net counts: 0.000
PSF-corrected count-rate upper limit: 0.000e+00 counts/s


In [12]:
from astropy.io import fits
evt = fits.open("detections/30892/578_evt2_filtered.fits")
print(evt["EVENTS"].columns.names)

with fits.open("detections/30892/578_evt2_filtered.fits") as hdul:
    ccd_ids = np.unique(hdul["EVENTS"].data["ccd_id"])
print("Active CCDs:", ccd_ids)

['time', 'ccd_id', 'node_id', 'expno', 'chipx', 'chipy', 'tdetx', 'tdety', 'detx', 'dety', 'x', 'y', 'pha', 'pha_ro', 'energy', 'pi', 'fltgrade', 'grade', 'status']
Active CCDs: [3 6 7 8]


In [25]:
# --- Require IMAGE regions so we don't silently misplace apertures ---
def parse_ds9_circles_require_image(filename):
    import re
    frame = None
    circles = []
    with open(filename) as f:
        for raw in f:
            t = raw.strip()
            if not t or t.startswith("#") or t.lower().startswith("global"):
                continue
            if t.lower() in ("image", "fk5"):
                frame = t.lower()
                continue
            if t.lower().startswith("circle("):
                nums = re.findall(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", t)
                if len(nums) >= 3:
                    circles.append(tuple(map(float, nums[:3])))
    if frame != "image":
        raise RuntimeError(
            f"{filename} is in {frame or 'unknown'} coords. "
            "Convert to IMAGE in DS9 (Region → Coordinate System → Image), then save."
        )
    if not circles:
        raise RuntimeError(f"No circle() found in {filename}")
    return circles

# Use it like this:
src = parse_ds9_circles_require_image(src_reg_file)[0]
bkg_regions = parse_ds9_circles_require_image(bkg_reg_file)

# --- Correct 1-sided upper bound for Poisson with known background ---
# Solve P(K <= N | mu) = alpha, with alpha = 1 - conf, where mu = s + b
# Works for any N (N=0 handled automatically)
import math

def poisson_cdf_leq(N, mu):
    # CDF = e^{-mu} sum_{k=0..N} mu^k/k^k!, computed stably by recursion
    # (OK for N up to a few hundred)
    if mu < 0:
        return 0.0
    term = math.exp(-mu)  # k=0
    s = term
    for k in range(1, N+1):
        term *= mu / k
        s += term
    return s

def upper_limit_net_counts(N_obs, b_expected, conf=0.90):
    alpha = 1.0 - conf
    # Find mu so that CDF(N_obs | mu) = alpha
    lo, hi = 0.0, max(1.0, N_obs + b_expected + 10.0)
    # Ensure hi is high enough (CDF decreases with mu)
    while poisson_cdf_leq(N_obs, hi) > alpha and hi < 1e5:
        hi *= 1.5
    # Bisection
    for _ in range(80):
        mid = 0.5 * (lo + hi)
        if poisson_cdf_leq(N_obs, mid) > alpha:
            lo = mid
        else:
            hi = mid
    mu_ul = 0.5 * (lo + hi)
    s_ul = max(0.0, mu_ul - b_expected)
    return s_ul

s_UL = upper_limit_net_counts(N_src, b_expected, conf=confidence)
rate_UL = s_UL / (exposure_time * psf_fraction)
print(f"\n{int(confidence*100)}% upper bound on net counts: {s_UL:.3f}")
print(f"PSF-corrected count-rate upper bound: {rate_UL:.3e} counts/s")



90% upper bound on net counts: 26.114
PSF-corrected count-rate upper bound: 1.451e-03 counts/s
